In [92]:
import requests as re
import foursquare
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library

In [93]:
#Open dataFrame with neighbourhoods
df=pd.read_csv('nbhoods_new.csv')
df.head(5)

,Borough,Neighborhood,LL,Bbox,Latitude,Longitude
0,Bronx,Wakefield,"-73.84720052054902,40.89470517661","[-73.84720052054902, 40.89470517661, -73.84720...",40.894705,-73.847201
1,Bronx,Co-op City,"-73.82993910812398,40.87429419303012","[-73.82993910812398, 40.87429419303012, -73.82...",40.874294,-73.829939
2,Bronx,Eastchester,"-73.82780644716412,40.887555677350775","[-73.82780644716412, 40.887555677350775, -73.8...",40.887556,-73.827806
3,Bronx,Fieldston,"-73.90564259591682,40.89543742690383","[-73.90564259591682, 40.89543742690383, -73.90...",40.895437,-73.905643
4,Bronx,Riverdale,"-73.9125854610857,40.890834493891305","[-73.9125854610857, 40.890834493891305, -73.91...",40.890834,-73.912585


In [94]:
#Check neighborhoods with similar names to make sure they are correct
df[df['Neighborhood'].str.contains('Chelsea')]

,Borough,Neighborhood,LL,Bbox,Latitude,Longitude
116,Manhattan,Chelsea M,"-74.00311633472813,40.744034706747975","[-74.00311633472813, 40.744034706747975, -74.0...",40.744035,-74.003116
244,Staten Island,Chelsea SI,"-74.1895604551969,40.59472602746295","[-74.1895604551969, 40.59472602746295, -74.189...",40.594726,-74.189560


In [1]:
#Check what kind of Bbox we have
print(df['Bbox'][4])#Seemd like we cannot use it as it has repeating pairs of coordinates

NameError: name 'df' is not defined

In [96]:
#Check names of all boroughs
df['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [97]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [98]:
#Plot the neighborhoods
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [99]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map - IS THAT POSSIBLE TO DO AREAS INSTEAD OF POINTS?
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [100]:
#Set up Foursquare - paste credentials
CLIENT_ID = FOURSQUARE_CLIENT_ID 
CLIENT_SECRET = FOURSQUARE_CLIENT_SECRET
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XYU3VK2M4ICPSGFHIFFYHX24NMD0OB0MTYCZAZKHSNLEUIUC
CLIENT_SECRET:4WMKKQZI5U5QUAXXXOWZTUKB5ALEMUCH1YM5QK5TJ4RUHC1F


In [101]:
#Get data for MANHATTAN
manhattan_data=df.loc[df['Borough']=='Manhattan'].copy()

In [102]:
manhattan_data

,Borough,Neighborhood,LL,Bbox,Latitude,Longitude
6,Manhattan,Marble Hill,"-73.91065965862981,40.87655077879964","[-73.91065965862981, 40.87655077879964, -73.91...",40.876551,-73.910660
100,Manhattan,Chinatown,"-73.99427936255978,40.71561842231432","[-73.99427936255978, 40.71561842231432, -73.99...",40.715618,-73.994279
101,Manhattan,Washington Heights,"-73.93690027985234,40.85190252555305","[-73.93690027985234, 40.85190252555305, -73.93...",40.851903,-73.936900
102,Manhattan,Inwood,"-73.92121042203897,40.86768396449915","[-73.92121042203897, 40.86768396449915, -73.92...",40.867684,-73.921210
103,Manhattan,Hamilton Heights,"-73.94968791883366,40.823604284811935","[-73.94968791883366, 40.823604284811935, -73.9...",40.823604,-73.949688
104,Manhattan,Manhattanville,"-73.9573853935188,40.8169344294978","[-73.9573853935188, 40.8169344294978, -73.9573...",40.816934,-73.957385
105,Manhattan,Central Harlem,"-73.94321112603905,40.81597606742414","[-73.94321112603905, 40.81597606742414, -73.94...",40.815976,-73.943211
106,Manhattan,East Harlem,"-73.94418223148524,40.79224946663033","[-73.94418223148524, 40.79224946663033, -73.94...",40.792249,-73.944182
107,Manhattan,Upper East Side,"-73.96050763135,40.775638573301805","[-73.96050763135, 40.775638573301805, -73.9605...",40.775639,-73.960508
108,Manhattan,Yorkville,"-73.94711784471826,40.775929849884875","[-73.94711784471826, 40.775929849884875, -73.9...",40.775930,-73.947118


In [103]:
manhattan_data=manhattan_data.reset_index(drop=True)

In [104]:
manhattan_data.loc[0, 'Latitude'] 

40.87655077879964

In [105]:
#Define variables for the function
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [106]:
#Define parameters for the function
radius = 908 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XYU3VK2M4ICPSGFHIFFYHX24NMD0OB0MTYCZAZKHSNLEUIUC&client_secret=4WMKKQZI5U5QUAXXXOWZTUKB5ALEMUCH1YM5QK5TJ4RUHC1F&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=908&limit=100'

In [107]:
results = requests.get(url=url).json()
results

{'meta': {'code': 200, 'requestId': '614833ec293a2f0285fd1bd0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 71,
  'suggestedBounds': {'ne': {'lat': 40.88472278697165,
    'lng': -73.89987203850542},
   'sw': {'lat': 40.86837877062763, 'lng': -73.9214472787542}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4baf59e8f964a520a6f93be3',
       'name': 'Bikram Yoga',
       'location': {'address': '5500 Broadway',
        'crossStreet': '230th Street',
        'lat': 40.876843690797934,
        'lng': -73.90620384

In [108]:
#def getNearbyVenues(names, latitudes, longitudes, radius=500):
    '''
    Returns the dataframe with neighborhood, venue name, venue lat, venue lon, venue category.

            Parameters:
                    names (list): string
                    latitudes (list): float
                    longitudes (list): float

            Returns:
                    nearby_venues (dataFrame): dataFrame
    '''

IndentationError: unexpected indent (<ipython-input-108-b29f3fe97d47>, line 2)

In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#       print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
#           lat, 
#           lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
#                 'Neighborhood Latitude', 
#                 'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [110]:
#Get Manhattan data
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

In [111]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3271, 5)


,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,Starbucks,40.877531,-73.905582,Coffee Shop


In [114]:
#Get data for BRONX
bronx_data=df.loc[df['Borough']=='Bronx'].copy()

In [115]:
bronx_data

,Borough,Neighborhood,LL,Bbox,Latitude,Longitude
0,Bronx,Wakefield,"-73.84720052054902,40.89470517661","[-73.84720052054902, 40.89470517661, -73.84720...",40.894705,-73.847201
1,Bronx,Co-op City,"-73.82993910812398,40.87429419303012","[-73.82993910812398, 40.87429419303012, -73.82...",40.874294,-73.829939
2,Bronx,Eastchester,"-73.82780644716412,40.887555677350775","[-73.82780644716412, 40.887555677350775, -73.8...",40.887556,-73.827806
3,Bronx,Fieldston,"-73.90564259591682,40.89543742690383","[-73.90564259591682, 40.89543742690383, -73.90...",40.895437,-73.905643
4,Bronx,Riverdale,"-73.9125854610857,40.890834493891305","[-73.9125854610857, 40.890834493891305, -73.91...",40.890834,-73.912585
5,Bronx,Kingsbridge,"-73.90281798724604,40.88168737120521","[-73.90281798724604, 40.88168737120521, -73.90...",40.881687,-73.902818
7,Bronx,Woodlawn,"-73.86731496814176,40.89827261213805","[-73.86731496814176, 40.89827261213805, -73.86...",40.898273,-73.867315
8,Bronx,Norwood,"-73.8793907395681,40.87722415599446","[-73.8793907395681, 40.87722415599446, -73.879...",40.877224,-73.879391
9,Bronx,Williamsbridge,"-73.85744642974207,40.88103887819211","[-73.85744642974207, 40.88103887819211, -73.85...",40.881039,-73.857446
10,Bronx,Baychester,"-73.83579759808117,40.866858107252696","[-73.83579759808117, 40.866858107252696, -73.8...",40.866858,-73.835798


In [116]:
bronx_data=bronx_data.reset_index(drop=True)

In [117]:
bronx_data.loc[0, 'Latitude'] 

40.894705176609996

In [118]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

In [119]:
print(bronx_venues.shape)
bronx_venues.head()

(1210, 5)


,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,Cooler Runnings Jamaican Restaurant,40.898083,-73.850259,Caribbean Restaurant


In [120]:
#Get venues for Brooklyn
brooklyn_data=df.loc[df['Borough']=='Brooklyn'].copy()

In [121]:
brooklyn_data=brooklyn_data.reset_index(drop=True)

In [122]:
brooklyn_data.loc[0, 'Latitude'] 

40.625801065010656

In [123]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

In [124]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2773, 5)


,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,Georgian Dream Cafe and Bakery,40.625586,-74.030196,Caucasian Restaurant
3,Bay Ridge,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
4,Bay Ridge,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot


In [125]:
#Get venues for Queens
queens_data=df.loc[df['Borough']=='Queens'].copy()

In [126]:
queens_data=queens_data.reset_index(drop=True)

In [127]:
queens_data.loc[0, 'Latitude'] 

40.76850859335492

In [128]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

In [129]:
print(queens_venues.shape)
queens_venues.head()

(2149, 5)


,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,Simply Fit Astoria,40.769114,-73.912403,Gym


In [130]:
#Get venues for Staten Island
staten_island_data=df.loc[df['Borough']=='Staten Island'].copy()

In [131]:
staten_island_data=staten_island_data.reset_index(drop=True)

In [132]:
staten_island_data.loc[0, 'Latitude'] 

40.6449815710044

In [133]:
staten_island_venues = getNearbyVenues(names=staten_island_data['Neighborhood'],
                                   latitudes=staten_island_data['Latitude'],
                                   longitudes=staten_island_data['Longitude']
                                  )

In [134]:
print(staten_island_venues.shape)
staten_island_venues.head()

(833, 5)


,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,A&S Pizzeria,40.643940,-74.077626,Pizza Place
1,St. George,Beso,40.643306,-74.076508,Tapas Restaurant
2,St. George,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
3,St. George,Shake Shack,40.643660,-74.075891,Burger Joint
4,St. George,Enoteca Maria,40.641941,-74.077320,Italian Restaurant


In [135]:
#Create dataframe for all neighborhoods
foursquare_venues_new=pd.concat([manhattan_venues, brooklyn_venues, bronx_venues, queens_venues, staten_island_venues], axis=0, ignore_index=True)

In [136]:
#Check 'tricky' neighborhoods just in case
foursquare_venues_new[foursquare_venues_new['Neighborhood'].str.contains('Chelsea')]

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
1281,Chelsea M,Milk & Hops Chelsea,40.744751,-74.002595,Beer Bar
1282,Chelsea M,Atlantic Theater Company (Linda Gross Theater),40.743953,-74.001476,Theater
1283,Chelsea M,La Bergamote,40.744733,-74.002729,Café
1284,Chelsea M,TAO Downtown,40.742539,-74.003812,Asian Restaurant
1285,Chelsea M,Billy's Bakery,40.745374,-74.002119,Cupcake Shop
1286,Chelsea M,Bathtub Gin,40.743638,-74.003290,Speakeasy
1287,Chelsea M,Buddakan,40.742120,-74.004612,Chinese Restaurant
1288,Chelsea M,Mojo Omakase,40.744895,-74.002700,Sushi Restaurant
1289,Chelsea M,Worthwild,40.744450,-74.003204,New American Restaurant
1290,Chelsea M,The Joyce Theater,40.742829,-74.000534,Theater


In [137]:
#Write parsed data to the file to share with the team
foursquare_venues_new.to_csv('foursquare_venues_new.csv')